Ici on récupère les données via PISTE l'API permettant d'avoir accès aux données de Légifrance, le code de connexion OAuth est donné sur le Gitlab de Piste à cette [adresse](https://gitlab.com/piste_lab/oauth_connectors/-/blob/master/Python/Oauth2ClientCredentialsTest.py?ref_type=heads). Il est tout de même nécessaire d'installer les modules du code suivant.

In [1]:
%pip install python-dotenv
%pip install requests-oauthlib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Pour pour envoyer des requêtes à l'API nous devons d'abord obtenir un token, après s'être inscrit sur le [site de PISTE](https://piste.gouv.fr/). Ce token est obtenu en envoyant une requête contenant mon identifiant client et mon code de client au site d'autorisation. Nous avons ensuite une autorisation d'une heure avec ce token pour exploiter l'API de Légifrance. 

In [4]:
TOKEN_URL = "https://oauth.piste.gouv.fr/api/oauth/token"

import os
from dotenv import load_dotenv
import requests
from requests_oauthlib import OAuth2Session

# Cette partie sert à récupérer les ID de connexions qui sont dans un fichier .env que l'on doit créer à chaque fois que l'on copie le repository github, cela permet de les conserver secret (à mettre dans le read.me)
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# Requette d'un token pour avoir accès à l'API 
res = requests.post(
  TOKEN_URL,
  data={
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "openid"
  }
)

# Récupération du token 
token = res.json()
client = OAuth2Session(client_id, token=token)

Maintenant en utilisant le lien d'exploitation de l'API auquel on rajoute l'endpoints qui permet d'accéder à ce que l'on veut faire nous récupérons les données. La liste des endpoints pour l'API de Légifrance est disponible [ici](https://piste.gouv.fr/index.php?option=com_apiportal&view=apitester&usage=api&apitab=tests&apiName=L%C3%A9gifrance&apiId=7e5a0e1d-ffcc-40be-a405-a1a5c1afe950&managerId=3&type=rest&apiVersion=2.4.2&Itemid=179&swaggerVersion=2.0&lang=fr).

In [6]:
API_HOST = "https://api.piste.gouv.fr/dila/legifrance/lf-engine-app"

Le code ci-dessous permet de récupérer un ensemble de documents appartenant à l'ensemble des lois, ordonnance, décrets et arrêtés de Légifrance entre le premier janvier 1996 et le 31 août 2022 qui contiennent au moins un des mots d'une liste définie. Cette liste comprend les termes des taux de délinquances détaillés [ici](database_délinquance.ipynb#Calcul-des-taux-de-délinquance). Ce code est un json et sera envoyé par une méthode post à l'API, elle nous renverra un nombre de résultats limités (100 ici). Ce code est très inspiré de celui disponible sur le [site de l'API](https://piste.gouv.fr/index.php?option=com_apiportal&view=apitester&usage=api&apitab=tests&apiName=L%C3%A9gifrance&apiId=7e5a0e1d-ffcc-40be-a405-a1a5c1afe950&managerId=3&type=rest&apiVersion=2.4.2&Itemid=179&swaggerVersion=2.0&lang=fr) à l'endpoint /search.

In [15]:
import requests
import pandas as pd
import math



code_api_LODA = {
  "recherche": {
    "filtres": [
      {
        "dates": {
          "start": "1996-01-01",
          "end": "2022-08-31"
        },
        "facette": "DATE_SIGNATURE"
      }
    ],
    "sort": "SIGNATURE_DATE_DESC",
    "fromAdvancedRecherche": True,
    "champs": [
      {
        "criteres": [
          {
            "valeur": "délinquance",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "crime",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "délit",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Homicides",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Vols",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Stupéfiants",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Escroquerie",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Contrefaçon",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Sequestrations",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Recels",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Proxénétisme",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Menaces",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Cambriolages",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "infraction",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Attentats",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "dégradations",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Outrages",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          }
        ],
        "operateur": "OU",
        "typeChamp": "ALL"
      }
    ],
    "pageSize": 100,
    "pageNumber": 1,
    "operateur": "ET",
    "typePagination": "DEFAUT"
  },
  "fond":"LODA_DATE"
}

code_api_CODE = {
    "recherche": {   
        "champs": [
            {
                "typeChamp": "ARTICLE",
                "criteres": [
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "délinquance",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "crime",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "délit",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Homicides",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Vols",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Stupéfiants",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Escroquerie",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Contrefaçon",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Sequestrations",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Recels",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Proxénétisme",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Menaces",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Cambriolages",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "infraction",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Attentats",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "dégradations",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Outrages",
                        "operateur": "OU"
                    }
                ],
                "operateur": "OU"
            }
        ],
        "pageNumber": 1,
        "pageSize": 10,
        "operateur": "ET",
        "sort": "DATE",
        "typePagination": "DEFAUT"
    },
    "fond": "CODE_DATE"
}




On définit une fonction qui nous permet de récupérer tous les documents de notre recherche, car chaque recherche est limitée à une page de 100 éléments.

In [8]:

def collect_all_results(client, api_host, code):
    # Récupérer le total de résultats et calculer le nombre de pages
    response = client.post(api_host, json=code).json()
    total_results = response.get("totalResultNumber", 0)
    page_size = code["recherche"]["pageSize"]
    total_pages = math.ceil(total_results / page_size)

    print(f"Total de résultats : {total_results}")
    print(f"Nombre de pages à récupérer : {total_pages}")

    # Liste pour stocker tous les résultats
    all_results = []
    for page_number in range(1, total_pages + 1):
        print(f"Récupération de la page {page_number}/{total_pages}...")
        code["recherche"]["pageNumber"] = page_number
        response = client.post(api_host, json=code).json()
        all_results.extend(response.get("results", []))

    return all_results


In [ ]:
all_results_LODA = collect_all_results(client, API_HOST+"/search", code_api_LODA)
all_results_CODE = collect_all_results(client, API_HOST+"/search", code_api_CODE)


Total de résultats : 14874
Nombre de pages à récupérer : 149
Récupération de la page 1/149...
Récupération de la page 2/149...
Récupération de la page 3/149...
Récupération de la page 4/149...
Récupération de la page 5/149...
Récupération de la page 6/149...
Récupération de la page 7/149...
Récupération de la page 8/149...
Récupération de la page 9/149...
Récupération de la page 10/149...
Récupération de la page 11/149...
Récupération de la page 12/149...
Récupération de la page 13/149...
Récupération de la page 14/149...
Récupération de la page 15/149...
Récupération de la page 16/149...
Récupération de la page 17/149...
Récupération de la page 18/149...
Récupération de la page 19/149...
Récupération de la page 20/149...
Récupération de la page 21/149...
Récupération de la page 22/149...
Récupération de la page 23/149...
Récupération de la page 24/149...
Récupération de la page 25/149...
Récupération de la page 26/149...
Récupération de la page 27/149...
Récupération de la page 28/149

In [9]:

def results_to_dataframe(results):
    data = []
    for result in results:
        title = result.get("titles", [{}])[0].get("title") if result.get("titles") else None
        date = result.get("date")
        id = result.get("titles", [{}])[0].get("id")
        data.append({"Titre": title, "Date": date, "ID": id})
    return pd.DataFrame(data)


In [41]:

df_LODA = results_to_dataframe(all_results_LODA)
df_CODE = results_to_dataframe(all_results_CODE)

In [42]:
df_LODA = df_LODA.drop_duplicates()
df_LODA.to_csv("resultats_legifrance_loda.csv", index=False)

In [14]:
df_CODE["date_version"] = pd.to_datetime(df["ID"].str.split("_").str[-1], format="%d-%m-%Y")
start_date = "1996-01-01"
end_date = "2022-08-31"
df_CODE = df_CODE[(df_CODE["date_version"] >= pd.to_datetime(start_date)) & (df_CODE["date_version"] <= pd.to_datetime(end_date))]
df_CODE.to_csv("resultats_legifrance_code.csv", index=False)

NameError: name 'pd' is not defined

In [ ]:

code_api_JOFR = {
    "recherche": {   
        "champs": [
            {
                "typeChamp": "ARTICLE",
                "criteres": [
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "délinquance",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "crime",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "délit",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Homicides",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Vols",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Stupéfiants",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Escroquerie",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Contrefaçon",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Sequestrations",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Recels",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Proxénétisme",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Menaces",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Cambriolages",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "infraction",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Attentats",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "dégradations",
                        "operateur": "OU"
                    },
                    {
                        "typeRecherche": "UN_DES_MOTS",
                        "valeur": "Outrages",
                        "operateur": "OU"
                    }
                ],
                "operateur": "OU"
            }
        ],
        "pageNumber": 1,
        "pageSize": 10,
        "operateur": "ET",
        "sort": "DATE",
        "typePagination": "DEFAUT"
    },
    "fond": "JOFR_DATE"
}




client.post(API_HOST+"/search", json = code_api_JOFR).json()

SyntaxError: invalid syntax (4209379448.py, line 4)

In [ ]:
import json  # Pour gérer la sérialisation JSON

# Définir les données
code_api = {
    "recherche": {
        "champs": [
            {
                "typeChamp": "ARTICLE",
                "criteres": [
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "délinquance", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "crime", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "délit", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Homicides", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Vols", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Stupéfiants", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Escroquerie", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Contrefaçon", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Sequestrations", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Recels", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Proxénétisme", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Menaces", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Cambriolages", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "infraction", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Attentats", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "dégradations", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Outrages", "operateur": "OU"}
                ],
                "operateur": "OU"
            }
        ],
        "pageNumber": 1,
        "pageSize": 10,
        "operateur": "ET",
        "sort": "DATE",
        "typePagination": "DEFAUT"
    },
    "fond": "ALL"
}

code_api_2={
    "recherche": {
        "filtres": [
            {
                "dates": {
                    "start": "1996-01-01",
                    "end": "2022-08-31"
                },
                "facette": "DATE_SIGNATURE"
            }
        ],
        "champs": [
            {
                "typeChamp": "ARTICLE",
                "criteres": [
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "délinquance", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "crime", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "délit", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Homicides", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Vols", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Stupéfiants", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Escroquerie", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Contrefaçon", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Sequestrations", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Recels", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Proxénétisme", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Menaces", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Cambriolages", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "infraction", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Attentats", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "dégradations", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Outrages", "operateur": "OU"}
                ],
                "operateur": "OU"
            }
        ],
        "pageNumber": 1,
        "pageSize": 10,
        "operateur": "ET",
        "sort": "DATE",
        "typePagination": "DEFAUT"
    },
    "fond": "ALL"
}



results = client.post(API_HOST+"/search", json = code_api).json()
print(results)

In [12]:
from datetime import datetime

def is_within_date_range(section, start_date, end_date):
    """
    Vérifie si une section est dans la plage de dates spécifiée.

    :param section: Une section contenant les informations de date.
    :param start_date: Date de début (format YYYY-MM-DD).
    :param end_date: Date de fin (format YYYY-MM-DD).
    :return: True si la section est dans la plage de dates, False sinon.
    """
    date_debut_str = section.get('extracts', [{}])[0].get('dateDebut')
    if date_debut_str:
        date_debut = datetime.strptime(date_debut_str.split("T")[0], "%Y-%m-%d")
        return start_date <= date_debut <= end_date
    return False

def collect_all_results_1(client, api_host, code):
    # Récupérer le total de résultats et calculer le nombre de pages
    response = client.post(api_host, json=code).json()
    total_results = response.get("totalResultNumber", 0)
    page_size = code["recherche"]["pageSize"]
    total_pages = math.ceil(total_results / page_size)

    print(f"Total de résultats : {total_results}")
    print(f"Nombre de pages à récupérer : {total_pages}")

    # Liste pour stocker tous les résultats filtrés
    all_results = []

    # Définir la plage de dates
    start_date = datetime.strptime("1996-01-01", "%Y-%m-%d")
    end_date = datetime.strptime("2022-08-31", "%Y-%m-%d")

    for page_number in range(1, total_pages + 1):
        print(f"Récupération de la page {page_number}/{total_pages}...")
        code["recherche"]["pageNumber"] = page_number
        response = client.post(api_host, json=code).json()
        page_results = response.get("results", [])

        # Appliquer le filtre sur les résultats de la page courante
        for result in page_results:
            if any(is_within_date_range(section, start_date, end_date) for section in result.get('sections', [])):
                all_results.append(result)

    return all_results


In [16]:
results = collect_all_results_1(client, API_HOST+"/search", code_api)

Total de résultats : 1501517
Nombre de pages à récupérer : 150152
Récupération de la page 1/150152...
Récupération de la page 2/150152...
Récupération de la page 3/150152...
Récupération de la page 4/150152...
Récupération de la page 5/150152...
Récupération de la page 6/150152...
Récupération de la page 7/150152...
Récupération de la page 8/150152...
Récupération de la page 9/150152...
Récupération de la page 10/150152...
Récupération de la page 11/150152...
Récupération de la page 12/150152...
Récupération de la page 13/150152...
Récupération de la page 14/150152...
Récupération de la page 15/150152...
Récupération de la page 16/150152...
Récupération de la page 17/150152...
Récupération de la page 18/150152...
Récupération de la page 19/150152...
Récupération de la page 20/150152...
Récupération de la page 21/150152...
Récupération de la page 22/150152...
Récupération de la page 23/150152...
Récupération de la page 24/150152...
Récupération de la page 25/150152...
Récupération de la

KeyboardInterrupt: 